In [29]:
import dask.dataframe as dd
import pandas as pd

In [30]:
dfA = pd.DataFrame(
    {"x": ["x1", "x2", "x2", "x1", "x3", "x2"], "y": ["A", "B", "C", "B", "D", "E"]}
)
ddfA = dd.from_pandas(dfA, npartitions=2)

In [31]:
gA = ddfA.groupby("x").y.agg(list).reset_index()

In [33]:
dfB = pd.DataFrame({"x": ["x1", "x2", "x3"], "z": ["U", "V", "W"]})
ddfB = dd.from_pandas(dfB, npartitions=2)

In [34]:
ddfB.compute()

,x,z
0,x1,U
1,x2,V
2,x3,W


In [36]:
print(gA.merge(ddfB, on="x", how="left").compute())

    x          y  z
0  x1     [A, B]  U
1  x2  [B, C, E]  V
2  x3        [D]  W


In [37]:
d = {0: [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 
         1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0], 
     1: [16, 1, 0, 15, 7, 14, 13, 7, 3, 10, 14, 8, 10, 3,  5, 4, 8, 13, 6, 2, 
         11, 11, 0, 12, 9, 16, 12, 15, 9, 1, 4, 5, 6, 2], 
     2: [32, 32, 32, 32, 32, 33, 46, 64, 97, 97, 99, 99, 100, 101, 101, 102, 103, 
         103, 103, 104, 105, 105, 106, 108, 109, 109, 110, 111, 111, 111, 112, 112, 121, 122]}
print (pd.DataFrame(d).pivot(1,0,2).applymap(chr).agg(''.join))

0
0    jozefeg@gmail.com
1      happy coding!  
dtype: object


In [38]:
import pandas as pd
import dask.dataframe as dd

df = pd.DataFrame({'c0': ['a', 'b', 'a', 'a', 'b'],
                   'c1': [1, 1, 1, 2, 5],
                   'c2': [0, 1, 0, 2, 5],
                  })
ddf = dd.from_pandas(df, 3)

op = lambda x: x.groupby("c1").mean().reset_index()

In [39]:
op

<function __main__.<lambda>(x)>

In [41]:
ddf.groupby("c1").mean().reset_index().compute()

,c1,c2
0,1,0.333333
1,2,2.000000
2,5,5.000000


In [42]:
ddf.groupby("c0").apply(op, meta={"c1": float, "c2": float}).reset_index().drop(columns="level_1")


ValueError: Metadata inference failed in `drop_by_shallow_copy`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
KeyError("['level_1'] not found in axis")

Traceback:
---------
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/dask/dataframe/utils.py", line 176, in raise_on_meta_error
    yield
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/dask/dataframe/core.py", line 5669, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/dask/dataframe/utils.py", line 712, in drop_by_shallow_copy
    df2.drop(columns=columns, inplace=True, errors=errors)
  File "/Us

In [43]:
ddf.groupby("c0").compute()

AttributeError: 'Column not found: compute'

In [45]:
ddf.groupby("c0").apply(op).reset_index().drop(columns="level_1").compute()


/var/folders/d2/116lnkgd0l7f51xr7msb2jnh0000gn/T/ipykernel_13981/2952609141.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf.groupby("c0").apply(op).reset_index().drop(columns="level_1").compute()


,c0,c1,c2
0,b,1,1.0
1,b,5,5.0
0,a,1,0.0
1,a,2,2.0


In [46]:
ddf.groupby("c0").apply(op).reset_index().compute()

/var/folders/d2/116lnkgd0l7f51xr7msb2jnh0000gn/T/ipykernel_13981/1304195809.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf.groupby("c0").apply(op).reset_index().compute()


,c0,level_1,c1,c2
0,b,0,1,1.0
1,b,1,5,5.0
0,a,0,1,0.0
1,a,1,2,2.0


In [48]:
ddf.groupby("c0").mean().groupby("c1").mean().reset_index().compute()

,c1,c2
0,1.333333,0.666667
1,3.000000,3.000000
